<a href="https://colab.research.google.com/github/ulil-azmy/FMHYedit/blob/main/gabung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **stil int ororesf text**

In [ ]:
import requests
import json



class JobPortalAPI:
    def __init__(self, name, url, method, headers=None, data=None, params=None, structure=None, custom_field=None):
        self.name = name
        self.url = url
        self.method = method
        self.headers = headers or {}
        self.data = data or {}
        self.params = params or {}
        self.structure = structure or {}
        self.custom_field = custom_field or {}

    def construct_request_url(self, params):
        # Construct the request URL based on the parameters provided
        if self.method == "GET":
            return self.url + '?' + '&'.join([f"{key}={value}" for key, value in params.items()])
        else:
            # Handle construction of the request URL for other HTTP methods if needed
            return self.url  # Modify this based on your API endpoint structure

    # (Other methods remain the same)
    def generate_dynamic_params(self, specific_condition):
        # Generate params based on specific conditions or variables
          dynamic_params = {}

          if specific_condition:
              # Add parameters based on the condition
              dynamic_params['key'] = 'value'

          return dynamic_params

    def make_request(self, override_params=None):
        final_url = None  # Initialize final_url here

        try:
            if self.method == "GET":
                if override_params:
                    self.params.update(override_params)

                final_url = requests.Request('GET', self.url, headers=self.headers, params=self.params).prepare().url
                response = requests.get(self.url, headers=self.headers, params=self.params)
            elif self.method == "POST":
                response = requests.post(self.url, headers=self.headers, json=self.data)
            else:
                raise ValueError(f"Unsupported HTTP method for {self.name}")

            print(f"Final URL: {final_url}")
            return response
        except requests.RequestException as e:
            print(f"Request to {self.name} failed:", str(e))
            return None


# Rest of your code remains unchanged..


    def load_job_portals_configuration():
        # Load configuration from a file or database
        return [
            {
        "name": "Glints",
        "url": "https://glints.com/api/v2/graphql",
        "method": "POST",  # <-- Specify the method here for Glints
        "headers": {
          "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
        },
        "data": {
            "operationName": "searchJobs",
            "variables": {
                "data": {

                      "SearchTerm": "\"data analyst\"",
                      "CountryCode": "ID",
                      "limit": 1,
                      "sortBy": "LATEST",
                      "offset": 0,
                      "includeExternalJobs": True,
                      "sources": ["NATIVE", "SUPER_POWERED"],
                      "searchVariant": "VARIANT_A"
                }
            },
          "query": "query searchJobs($data: JobSearchConditionInput!) { searchJobs(data: $data) { jobsInPage { id title isRemote status createdAt updatedAt isActivelyHiring isHot shouldShowSalary educationLevel type fraudReportFlag salaryEstimate { minAmount maxAmount CurrencyCode } company { ...CompanyFields } citySubDivision { id name } city { ...CityFields } country { ...CountryFields } salaries { ...SalaryFields } location { ...LocationFields } minYearsOfExperience maxYearsOfExperience source type hierarchicalJobCategory { id level name children { name level id __typename } parents { id level name __typename } __typename } skills { skill { id name __typename } mustHave __typename } __typename } numberOfJobsCreatedInLast14Days totalJobs __typename } } fragment CompanyFields on Company { id name logo status IndustryId __typename } fragment CityFields on City { id name __typename } fragment CountryFields on Country { code name __typename } fragment SalaryFields on JobSalary { id salaryType salaryMode maxAmount minAmount CurrencyCode __typename } fragment LocationFields on HierarchicalLocation { id name administrativeLevelName formattedName level slug parents { id name administrativeLevelName formattedName level slug parents { level formattedName slug __typename } __typename } __typename }"
        },
        "custom_field": {
                "total_jobs": "data.searchJobs.totalJobs",
                "page_limit":30,
                "company_name": "data.searchJobs.jobsInPage.0.company.name"
            }
            },
            {

            "name": "Jobstreet",
            "url": "https://www.jobstreet.co.id/api/chalice-search/v4/search",
            "method": "GET",  # Specify POST for Glints
            "params": {
                "siteKey": "ID-Main",
                "page": 1,
                "keywords": "\"data+analyst\"",
                "pageSize": 1,
                "locale": "id-ID",
                "sortmode":"ListedDate"
            },

          "custom_field": {
                "total_jobs": "totalCount",
                "page_limit":30,
                "company_name": "data.0.companyName",
            }
            },
            {
            "name": "Kalibrr",
            #sample final api endpoint: https://www.kalibrr.com/kjs/job_board/search?limit=403&offset=0&country=Indonesia&text=%22data+analyst%22&sort_direction=asc&sort_field=application_end_date
            "url": "https://www.kalibrr.com/kjs/job_board/search",
            "method": "GET",
            "params": {  # Renamed: Total records fetching params
                "limit": 1,
                "offset": 0,
                "country": "Indonesia",
                "text": "\"data+analyst\"",
                "sort_direction": "desc",
                "sort_field": "activation_date"
            },
            "custom_field": {  # Renamed: Data extraction params
                "total_jobs": "count",
                "page_limit":15,
                "company_name": "jobs.0.company.name"
            }
        }
            # Add more job portal configurations if needed...
        ]


def extract_total_jobs(response_json, total_jobs_key, page_size):
    keys = total_jobs_key.split('.')
    result = response_json
    for key in keys:
        result = result.get(key)
        if result is None:
            break

    if result is None:
        print(f"Total jobs value for key '{total_jobs_key}' is None.")
        return None, None, None

    try:
        total_jobs = int(result)
    except (ValueError, TypeError):
        print(f"Cannot convert '{result}' to integer for '{total_jobs_key}' key.")
        return None, None, None

    total_pages = -1
    if page_size > 0:
        # Calculate total pages
        total_pages = -(-total_jobs // page_size)  # Ceiling division for total pages

    return total_jobs, total_pages, page_size


def calculate_offset(page_number, page_size):
    # Calculate the offset based on page number and page size
     return (page_number - 1) * page_size if page_number > 0 else 0

    # Usage example:
total_jobs, total_pages, page_size = extract_total_jobs(response_json, total_jobs_key, page_size)


if __name__ == "__main__":
    job_portals = load_job_portals_configuration()

    for portal_config in job_portals:
        portal = JobPortalAPI(**portal_config)
        response = portal.make_request()  # Initial request without offset

        total_jobs_key = portal_config.get('custom_field', {}).get('total_jobs')
        page_size = portal_config.get('custom_field', {}).get('page_limit')

        if total_jobs_key and page_size is not None:
            total_jobs, total_pages, _ = extract_total_jobs(response.json(), total_jobs_key, page_size)

            if total_jobs is not None:
                print(f"Total jobs from {portal.name}: {total_jobs}")
                print(f"Total Pages from {portal.name}: {total_pages}")
                print(f"Page Size {portal.name}: {page_size}")

                base_url = portal_config['url']
                for page_number in range(1, total_pages + 1):
                    offset = calculate_offset(page_number, page_size)
                    page_params = {"offset": offset, "limit": page_size, "page":page_number}  # Include the limit parameter
                    response = portal.make_request(override_params=page_params)

                   # Construct the request URL with the given parameters
                    request_url = portal.construct_request_url(page_params)
                    print(f"Request URL for Page {page_number}: {request_url}")

                    response = portal.make_request(override_params=page_params)

                    # Process the response for each page...
                    # Print the response for each page
                    print(f"Response for Page {page_number}:")
                    print(response.text)  # This prints the response content, adjust as needed

